논문 출처 : https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf

초록

statistical Language Modeling의 목표는 단어의 시퀀스에 대한 결합확률함수를 학습시키는 것임. 하지만 이는 모델의 테스트에 쓰이는 단어의 시퀀스는 다르기 때문에 발생하는 '차원의 저주' 때문에 어렵다. n-gram을 기반으로 전통적으로 성공적인 방법은 training 데이터에서 등장하는 짧은 중복 시퀀스들을 합쳐서 일반화를 하는 방법이다. 우리는 차원문제를 해결하기 위해 각 훈련 문장이 의미상으로 이웃하는 문장들의 exponential number를 모델에 전달할 수 있는 단어의 분산표현을 학습시키는 것을 제안한다. 이 모델은 (1) 각 단어의 분산 표현과 (2) 단어 시퀀스의 확률 함수를 동시에 학습한다. 이전에 한번도 나타나지 않은 단어들의 시퀀스들이 만약 이전에 학습된 문장의 단어들과 비슷한 단어들로 구성되었다면 높은 확률을 갖기에 일반화될 수 있다. 거대 모델들을 합리적인 시간 내에 학습시키는 것은 매우 어렵다. 우리는 신경망구조를 사용하여 실험을 하였는데 위의 방식으로 접근했을 때 향상된 n-gram model의 성능을 보여주었고 더 긴 문맥에서 이점이 있음을 발견했다.

 

1. 도입

- 차원의 저주는 모델의 학습을 어렵게 하는데 특히, 다양한 discrete random varaiables(문장 속 단어들 등) 사이에서 joint distribution(결합확률분포)을 모델링하려고 할 때 심하다. 예를 들어 사이즈가 10만인 Vocabulary의 10개의 연속적인 단어에 대해 joint distribution을 모델하려고 하면 100000^10-1 만큼의 파라미터가 필요하다. 연속 확률 분포라면 일반화가 더 쉽지만 이산 공간에서는 일반화의 구조가 명백하지 않다. 이러한 문제를 해결하기 위해 분산표현(distributed Representation)을 고안하였다. 분산 표현은 기존의 one-hot-vector 처럼 vocabulary size 전체를 벡터의 차원수로 두는 것이 아니라 훨씬 작은 m차원의 벡터로 표현하여 Dense vector로 나타낼 수 있게 한다. 이 때 분산표현은 이산분포가 아닌 연속 분포로서 smoothness 측면에서 효율적이며 벡터 간 유사도와 거리 계산이 가능하여 단어 간의 유사성을 표현하는 것이 가능하다.

어떤 문장에서 그 전에 나온 모든 단어들을 이용하여 다음 단어를 예측하는 것 보다 단어의 순서를 고려함과 동시에 가까운 단어들이 통계적으로 더욱 dependant하기에 이전의 n-1개의 단어들에 대한 조건부 확룔을 이용하는 것이 N-gram Model이다. 하지만 training 말뭉치에 없던 단어들의 조합이 새롭게 등장할 경우 이에 대하여서는 학습한 내용이 없기에 확률은 0이 된다. 이러한 문제를 해결하기 위해 더 작은 context를 살펴보거나 특정한 값을 더해 확률이 0이 되지 않게 하는 Smoothing 방법이 존재하지만 단어 간 유사성을 고려하기 위해 분산표현을 사용하였다.

 

2. A Neural Model

우선 Input layer에서는 n-1개의 단어들에 대한 One-hot-vector인 |V| x 1 크기의 벡터 w들과 |V| x m 크기의 행렬 C가 필요하다. 이 때 행렬 C는 Random initialized 행렬로서 이를 벡터 w와 곱함으로서 input layer의 결과물인 x가 도출된다. 이후 x는 Hidden layer와 Output layer를 다음의 식 처럼 거치게 된다.

 

y = b + Wx + U tanh(d + Hx)

 

W는 input layer와 output layer간의 direct connection을 만들 경우 weights 이며 b는 bias에 해당한다. U는 hidden layer에서 output layer로 갈 때의 weights, H는 hidden layer의 weights, d는 bias에 해당한다. 

 

이렇게 나온 y 는 각 output 단어에 대한 비정규화된 log-probabilites이기에 softmax 연산을 거치게 되고 이 결과들을 정답 테이블과 비교하여 back-propagation으로 학습이 이루어 진다. 이 때 확률적 경사하강법이 사용된다.

 

3. Parallel Implementation

 많은 계산을 효율적으로 하는 방법을 설명함. 최근에는 gpu의 성능이 좋아졌기에 넘어감.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

def make_batch():
    input_batch = []
    target_batch = []
    
    for sen in sentences:
        word = sen.split() #공백을 기준으로 토큰화
        input = [word_dict[n] for n in word[:-1]] #제일 끝 요소 빼고 input으로 넣음
        target = word_dict[word[-1]] #causal language modeling으로 제일 마지막 단어를 예측하도록 하는 모델
        
        input_batch.append(input)
        target_batch.append(target)
        
    return input_batch, target_batch

#Model
class NNLM(nn.Module):
    def __init__(self):
        super(NNLM, self).__init__() #super():상위 클래스 상속
        self.C = nn.Embedding(n_class, m)
        self.H = nn.Linear(n_step * m, n_hidden, bias=False) #bias=False를 사용하면 parameter의 수가 감소하여 모델 단순화 가능
        self.d = nn.Parameter(torch.ones(n_hidden)) #nn.Parameter : nn.Module 안에 만들어진 tensor들을 보관가능
        self.U = nn.Linear(n_hidden, n_class, bias=False)
        self.W = nn.Linear(n_step*m, n_class, bias=False)
        self.b = nn.Parameter(torch.ones(n_class))
        
    def forward(self,X):
        X = self.C(X) #X = [batch_size, n_step, m]
        X = X.view(-1, n_step*m)
        tanh = torch.tanh(self.d + self.H(X))
        output = self.b + self.W(X) + self.U(tanh)
        return output
    
if __name__ == '__main__': #main함수의 선언, 시작을 의미한다
    n_step = 2 #step의 수, n-1 in paper
    n_hidden = 2
    m = 2 #embedding size
    
    sentences = ['i like dog', 'i love coffee', 'i hate milk']
    
    word_list = ' '.join(sentences).split()
    word_list = list(set(word_list))
    word_dict = {w: i for i,w in enumerate(word_list)}
    number_dict = {i:w for i, w in enumerate(word_list)}
    n_class = len(word_dict) #vocabulary의 단어 수
    
    model = NNLM()
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=.001)
    
    input_batch, target_batch = make_batch()
    input_batch = torch.LongTensor(input_batch)
    target_batch = torch.LongTensor(target_batch)
    
    #학습 시작
    for epoch in range(5000):
        optimizer.zero_grad()
        output = model(input_batch)
        
        #output : [batch_size, n_class], target_batch : [batch_size]
        loss = criterion(output, target_batch)
        if (epoch +1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch +1), 'cost =', '{:.6f}'.format(loss))
        
        loss.backward()
        optimizer.step()
        
    #Prediction
    predict = model(input_batch).data.max(1, keepdim=True)[1]
    
    #Test
    print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])
        

Epoch: 1000 cost = 0.095607
Epoch: 2000 cost = 0.016130
Epoch: 3000 cost = 0.004702
Epoch: 4000 cost = 0.001830
Epoch: 5000 cost = 0.000830
[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
